In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Generate dataset with 2000 instances, 3 features, and 1 output
X, y = make_regression(n_samples=2000, n_features=3, n_targets=1, noise=0.1, random_state=42)

# Split the data: 1500 for training, 500 for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1500, test_size=500, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # unsqueeze for single output dimension
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create TensorDataset and DataLoader for training and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network architecture
class RegressionNN6Layers(nn.Module):
    def __init__(self):
        super(RegressionNN6Layers, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 4),  # Input layer
            nn.ReLU(),
            nn.Linear(4, 4),  # Hidden layer
            nn.ReLU(),
            nn.Linear(4, 4),  # Hidden layer
            nn.ReLU(),
            nn.Linear(4, 4),  # Hidden layer
            nn.ReLU(),
            nn.Linear(4, 4),  # Hidden layer
            nn.ReLU(),
            nn.Linear(4, 1)   # Output layer
        )

    def forward(self, x):
        return self.layers(x)

# Instantiate the model, define loss function and optimizer
model = RegressionNN6Layers()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training parameters
epochs = 100
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()           # Clear gradients
        outputs = model(inputs)         # Forward pass
        loss = criterion(outputs, targets)  # Calculate loss
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update weights
        running_loss += loss.item() * inputs.size(0)  # Accumulate loss
    
    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

# Switch to evaluation mode
model.eval()
predictions = []
actuals = []

with torch.no_grad():  # No need to compute gradients
    for inputs, targets in test_loader:
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().tolist())  # Append predictions
        actuals.extend(targets.squeeze().tolist())      # Append actual values

# Convert predictions and actuals to lists and calculate metrics
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R-squared: {r2:.4f}")

# Convert predictions and actuals to NumPy arrays for plotting
predictions = np.array(predictions)
actuals = np.array(actuals)

# Plot the predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.plot(actuals, label='Actual Values', linestyle='-', color='b', markersize=4)
plt.plot(predictions, label='Predicted Values',  linestyle='--', color='r', markersize=4)
plt.title('Model Predictions vs. Actual Values on Test Set')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()

# Define the file path
file_path = 'data/regression_test_6layers.txt'

# Combine features and output for each test instance
test_data = np.hstack((X_test, y_test.reshape(-1, 1)))

# Save to file without brackets
with open(file_path, 'w') as f:
    for row in test_data:
        #